**Jupyter notebook for test of any type**

In [3]:
import sys
import os
import pandas as pd
from datasets import load_dataset
import os
import numpy as np
import pyspark as ps
from pyspark.sql import SparkSession


# Get the absolute path of the project root
project_root = os.path.abspath("..")  # Adjust if necessary

# Add the src directory to Python's path
sys.path.append(os.path.join(project_root, "src"))

if project_root not in sys.path:
    sys.path.append(project_root)


In [2]:
from data import s3_utils as s3U

In [7]:
df = pd.read_csv("hf://datasets/Shadow-Blade/financialNews/financialNews.csv")

In [ ]:
os.makedirs('../data/raw/', exist_ok=True) 
df.to_csv('../data/raw/financialNews_shadow.csv')

In [3]:
bucket= "financialdata-sa"
news_external = "RawNews/All_external.csv"

In [4]:
news1 = s3U.get_csv_as_spark(bucket_name=bucket, s3_key=news_external)

your 131072x1 screen size is bogus. expect trouble
25/03/14 19:16:39 WARN Utils: Your hostname, Andres-F resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/14 19:16:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/14 19:16:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 19:16:42 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [ ]:
news1.show(5)

+--------------------+--------------------+------------+--------------------+-----------------+------+-------+-----------+------------+----------------+---------------+
|                Date|       Article_title|Stock_symbol|                 Url|        Publisher|Author|Article|Lsa_summary|Luhn_summary|Textrank_summary|Lexrank_summary|
+--------------------+--------------------+------------+--------------------+-----------------+------+-------+-----------+------------+----------------+---------------+
|2020-06-05 06:30:...|Stocks That Hit 5...|           A|https://www.benzi...|Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|
|2020-06-03 06:45:...|Stocks That Hit 5...|           A|https://www.benzi...|Benzinga Insights|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|
|2020-05-26 00:30:...|71 Biggest Movers...|           A|https://www.benzi...|       Lisa Levin|  NULL|   NULL|       NULL|        NULL|            NULL|   

In [ ]:
spark = SparkSession.builder \
    .appName("ConvertCSVtoParquet") \
    .getOrCreate()

df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("../data/raw/nasdaq_exteral_data.csv")